In [117]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Data = pd.read_csv("Anime.csv")
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18495 entries, 0 to 18494
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Rank             18495 non-null  int64  
 1   Name             18495 non-null  object 
 2   Japanese_name    7938 non-null   object 
 3   Type             18495 non-null  object 
 4   Episodes         9501 non-null   float64
 5   Studio           12018 non-null  object 
 6   Release_season   4116 non-null   object 
 7   Tags             18095 non-null  object 
 8   Rating           15364 non-null  float64
 9   Release_year     18112 non-null  float64
 10  End_year         2854 non-null   float64
 11  Description      18491 non-null  object 
 12  Content_Warning  1840 non-null   object 
 13  Related_Mange    7627 non-null   object 
 14  Related_anime    10063 non-null  object 
 15  Voice_actors     15309 non-null  object 
 16  staff            13005 non-null  object 
dtypes: float64(4

In [118]:
Data.isnull().sum()

Rank                   0
Name                   0
Japanese_name      10557
Type                   0
Episodes            8994
Studio              6477
Release_season     14379
Tags                 400
Rating              3131
Release_year         383
End_year           15641
Description            4
Content_Warning    16655
Related_Mange      10868
Related_anime       8432
Voice_actors        3186
staff               5490
dtype: int64

In [119]:
Data['Rating'] = Data['Rating'].fillna(Data['Rating'].mean())
Data['Rating'].isnull().sum()

0

In [120]:
Data = Data.fillna(0)
Data.isnull().sum()

Rank               0
Name               0
Japanese_name      0
Type               0
Episodes           0
Studio             0
Release_season     0
Tags               0
Rating             0
Release_year       0
End_year           0
Description        0
Content_Warning    0
Related_Mange      0
Related_anime      0
Voice_actors       0
staff              0
dtype: int64

In [121]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18495 entries, 0 to 18494
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Rank             18495 non-null  int64  
 1   Name             18495 non-null  object 
 2   Japanese_name    18495 non-null  object 
 3   Type             18495 non-null  object 
 4   Episodes         18495 non-null  float64
 5   Studio           18495 non-null  object 
 6   Release_season   18495 non-null  object 
 7   Tags             18495 non-null  object 
 8   Rating           18495 non-null  float64
 9   Release_year     18495 non-null  float64
 10  End_year         18495 non-null  float64
 11  Description      18495 non-null  object 
 12  Content_Warning  18495 non-null  object 
 13  Related_Mange    18495 non-null  object 
 14  Related_anime    18495 non-null  object 
 15  Voice_actors     18495 non-null  object 
 16  staff            18495 non-null  object 
dtypes: float64(4

In [122]:
Data["Type"] = Data["Type"].astype("category")
Data["Studio"] = Data["Studio"].astype("category")
Data["Release_season"] = Data["Release_season"].astype("category")
Data["Content_Warning"] = Data["Content_Warning"].astype("category")

# Convert float columns that should be int
Data["Episodes"] = Data["Episodes"].fillna(0).astype(int)
Data["Release_year"] = Data["Release_year"].fillna(0).astype(int)
Data["End_year"] = Data["End_year"].astype("Int64")  # Nullable int type

# Text columns
Data["Description"] = Data["Description"].astype(str)
Data["Voice_actors"] = Data["Voice_actors"].astype(str)
Data["staff"] = Data["staff"].astype(str)

In [123]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18495 entries, 0 to 18494
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Rank             18495 non-null  int64   
 1   Name             18495 non-null  object  
 2   Japanese_name    18495 non-null  object  
 3   Type             18495 non-null  category
 4   Episodes         18495 non-null  int32   
 5   Studio           18495 non-null  category
 6   Release_season   18495 non-null  category
 7   Tags             18495 non-null  object  
 8   Rating           18495 non-null  float64 
 9   Release_year     18495 non-null  int32   
 10  End_year         18495 non-null  Int64   
 11  Description      18495 non-null  object  
 12  Content_Warning  18495 non-null  category
 13  Related_Mange    18495 non-null  object  
 14  Related_anime    18495 non-null  object  
 15  Voice_actors     18495 non-null  object  
 16  staff            18495 non-null  object 

In [124]:
from sklearn.preprocessing import MultiLabelBinarizer

# Step 1: Fill NaNs with empty string
Data['Tags'] = Data['Tags'].fillna('')

# Step 2: Convert all entries to string and split by comma
Data['Tags_list'] = Data['Tags'].apply(lambda x: str(x).split(','))

# Step 3: Remove extra spaces and clean empty strings
Data['Tags_list'] = Data['Tags_list'].apply(lambda tags: [tag.strip() for tag in tags if tag.strip() != ''])

# Step 4: Apply MultiLabelBinarizer
mlb = MultiLabelBinarizer()
tags_encoded = mlb.fit_transform(Data['Tags_list'])

# Step 5: Convert to DataFrame and concatenate with original data
tags_df = pd.DataFrame(tags_encoded, columns=mlb.classes_)
Data = pd.concat([Data, tags_df], axis=1)



In [139]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 1: Join Tags list into a string
Data['Tags_str'] = Data['Tags_list'].apply(lambda x: ' '.join(x))

# Step 2: TF-IDF Vectorization
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(Data['Tags_str'])  # sparse matrix

# Step 3: Fit Nearest Neighbors Model
nn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=11)  # 1 (self) + 10 recommendations
nn_model.fit(tfidf_matrix)

# Step 4: Recommendation Function
def get_recommendations(title):
    # Get index of anime
    idx = Data[Data['Name'] == title].index[0]
    tfidf_vector = tfidf_matrix[idx]

    # Find nearest neighbors
    distances, indices = nn_model.kneighbors(tfidf_vector)

    # Return top 10 recommended anime (excluding self)
    recommended_indices = indices.flatten()[1:]  # Skip index 0 (same anime)
    return Data['Name'].iloc[recommended_indices]


In [141]:
get_recommendations("Death Note")

1116     Death Note Rewrite 1: Visions of a God
1276       Death Note Rewrite 2: L's Successors
2068              Neuro: Supernatural Detective
900                                  ID:INVADED
20      Code Geass: Lelouch of the Rebellion R2
64         Code Geass: Lelouch of the Rebellion
6059                    Descendants of Darkness
1246                      Inuyashiki: Last Hero
2482                            Angels of Death
5352                    Monster Special Edition
Name: Name, dtype: object

In [145]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Step 1: Fill missing descriptions
Data['Description'] = Data['Description'].fillna('')

# Step 2: TF-IDF on Descriptions
tfidf_desc = TfidfVectorizer(stop_words='english')
tfidf_matrix_desc = tfidf_desc.fit_transform(Data['Description'])  # Sparse matrix

# Step 3: Nearest Neighbors Model (for similarity)
nn_desc = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=11)
nn_desc.fit(tfidf_matrix_desc)

# Step 4: Recommendation Function
def get_desc_recommendations(title):
    idx = Data[Data['Name'] == title].index[0]
    tfidf_vector = tfidf_matrix_desc[idx]

    distances, indices = nn_desc.kneighbors(tfidf_vector)

    # Skip the input anime itself (first one)
    recommended_indices = indices.flatten()[1:]
    return Data['Name'].iloc[recommended_indices]


In [147]:
get_desc_recommendations("Death Note")


1116       Death Note Rewrite 1: Visions of a God
6507                                Suna no Akari
12342                             Soul Reviver PV
15740                         The God of Death OP
7776                  Momo: The Girl God of Death
14911                            gdgd men's party
18281                 CHARON Pluto’s lonesomeness
15597                           Shadowverse Flame
7534                                To Be Heroine
935      Persona 3 the Movie 4: Winter of Rebirth
Name: Name, dtype: object

In [155]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors

# Step 1: Convert 'Tags' column to list format
Data['Tags'] = Data['Tags'].apply(lambda x: str(x))  # In case of non-string entries
Data['Tags'] = Data['Tags'].apply(lambda x: [tag.strip() for tag in x.split(',')])

# Step 2: Encode the tags
mlb = MultiLabelBinarizer()
tag_matrix = mlb.fit_transform(Data['Tags'])  # Sparse format

# Step 3: Fit NearestNeighbors model
nn_tag = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=11)
nn_tag.fit(tag_matrix)

# Step 4: Recommendation function
def get_tag_recommendations(title):
    idx = Data[Data['Name'] == title].index[0]
    tag_vector = tag_matrix[idx]

    distances, indices = nn_tag.kneighbors(tag_vector)

    recommended_indices = indices.flatten()[1:]  # Skip the input anime itself
    return Data['Name'].iloc[recommended_indices]


In [165]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# --- PREPARE TAG VECTORS ---
Data['Tags'] = Data['Tags'].apply(lambda x: str(x))
Data['Tags'] = Data['Tags'].apply(lambda x: [tag.strip() for tag in x.split(',')])
mlb = MultiLabelBinarizer()
tag_matrix = mlb.fit_transform(Data['Tags'])

# --- PREPARE DESCRIPTION VECTORS ---
Data['Description'] = Data['Description'].fillna('')
tfidf = TfidfVectorizer(stop_words='english')
desc_matrix = tfidf.fit_transform(Data['Description'])

# --- HYBRID RECOMMENDATION FUNCTION ---
def get_hybrid_recommendations(title, weight_tag=0.5, weight_desc=0.5):
    idx = Data[Data['Name'] == title].index[0]
    
    # Compute similarity safely
    sim_tag = cosine_similarity(tag_matrix[idx:idx+1], tag_matrix).flatten()
    sim_desc = cosine_similarity(desc_matrix[idx:idx+1], desc_matrix).flatten()
    
    # Normalize
    sim_tag = (sim_tag - sim_tag.min()) / (sim_tag.max() - sim_tag.min() + 1e-9)
    sim_desc = (sim_desc - sim_desc.min()) / (sim_desc.max() - sim_desc.min() + 1e-9)

    # Hybrid
    hybrid_sim = weight_tag * sim_tag + weight_desc * sim_desc

    # Get top 10 similar anime
    top_indices = hybrid_sim.argsort()[::-1][1:11]
    return Data['Name'].iloc[top_indices]


In [167]:
get_hybrid_recommendations('Death Note')

1116                Death Note Rewrite 1: Visions of a God
1276                  Death Note Rewrite 2: L's Successors
1696                                   Detective Academy Q
920           Detective Conan Movie 24: The Scarlet Bullet
3953                                        Ayatsuri Sakon
10983                              Suteki Tantei Labyrinth
1538                          Kindaichi Shounen no Jikenbo
6882     Kindaichi Shounen no Jikenbo: Shinigami Byouin...
2459     Detective Conan: The Mysterious Written Invita...
3004     Detective Conan: City Exploration! Get the Ani...
Name: Name, dtype: object

In [169]:
Data.to_csv("cleaned_data.csv", index=False)


In [173]:
import pickle

# Save tag-based similarity matrix
with open("tag_cosine_sim.pkl", "wb") as f:
    pickle.dump(tag_cosine_sim, f)

# Save description-based similarity matrix
with open("desc_cosine_sim.pkl", "wb") as f:
    pickle.dump(cosine_sim_desc, f)  # ← correct name!

# Save hybrid similarity matrix
with open("hybrid_cosine_sim.pkl", "wb") as f:
    pickle.dump(hybrid_sim, f)  # ← correct if you used this name earlier
